Model


---



In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection import FasterRCNN

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

__all__ = ["create_model"]


def create_model(num_classes, min_size=300, max_size=500, backbone="mobile_net"):
    if backbone == "mobile_net":
        mobile_net = torchvision.models.mobilenet_v2(pretrained=True)
        ft_backbone = mobile_net.features
        ft_backbone.out_channels = 1280

    elif backbone == "vgg_11":
        vgg_net = torchvision.models.vgg11(pretrained=True)
        ft_backbone = vgg_net.features
        ft_backbone.out_channels = 512

    elif backbone == "vgg_13":
        vgg_net = torchvision.models.vgg13(pretrained=True)
        ft_backbone = vgg_net.features
        ft_backbone.out_channels = 512

    elif backbone == "vgg_16":
        vgg_net = torchvision.models.vgg13(pretrained=True)
        ft_backbone = vgg_net.features
        ft_backbone.out_channels = 512

    elif backbone == "vgg_19":
        vgg_net = torchvision.models.vgg19(pretrained=True)
        ft_backbone = vgg_net.features
        ft_backbone.out_channels = 512

    elif backbone == "resnet_18":
        resnet_net = torchvision.models.resnet18(pretrained=True)
        modules = list(resnet_net.children())[:-1]
        ft_backbone = nn.Sequential(*modules)
        ft_backbone.out_channels = 512

    elif backbone == "resnet_34":
        resnet_net = torchvision.models.resnet34(pretrained=True)
        modules = list(resnet_net.children())[:-1]
        ft_backbone = nn.Sequential(*modules)
        ft_backbone.out_channels = 512

    elif backbone == "resnet_50":
        resnet_net = torchvision.models.resnet50(pretrained=True)
        modules = list(resnet_net.children())[:-1]
        ft_backbone = nn.Sequential(*modules)
        ft_backbone.out_channels = 2048

    elif backbone == "resnet_101":
        resnet_net = torchvision.models.resnet101(pretrained=True)
        modules = list(resnet_net.children())[:-1]
        ft_backbone = nn.Sequential(*modules)
        ft_backbone.out_channels = 2048

    elif backbone == "resnet_152":
        resnet_net = torchvision.models.resnet152(pretrained=True)
        modules = list(resnet_net.children())[:-1]
        ft_backbone = nn.Sequential(*modules)
        ft_backbone.out_channels = 2048

    elif backbone == "resnext101_32x8d":
        resnet_net = torchvision.models.resnext101_32x8d(pretrained=True)
        modules = list(resnet_net.children())[:-1]
        ft_backbone = nn.Sequential(*modules)
        ft_backbone.out_channels = 2048

    else:
        print("Error Wrong unsupported Backbone")
        return

    ft_mean = [0.485, 0.456, 0.406]
    ft_std = [0.229, 0.224, 0.225]

    ft_anchor_generator = AnchorGenerator(
        sizes=((32, 64, 128),), aspect_ratios=((0.5, 1.0, 2.0),)
    )

    ft_roi_pooler = torchvision.ops.MultiScaleRoIAlign(
        featmap_names=['0','1','3'], output_size=7, sampling_ratio=2
    )

    ft_model = FasterRCNN(
        # min_size=ft_min_size,
        # max_size=ft_max_size,
        image_mean=ft_mean,
        image_std=ft_std,
        backbone=ft_backbone,
        num_classes=num_classes,
        rpn_anchor_generator=ft_anchor_generator,
        )

    return ft_model

Configurations

----

In [ ]:
TRAIN_CSV_PATH = "/content/drive/MyDrive/testing/csv_files/train_labels.csv"
VALIDATION_CSV_PATH = "/content/drive/MyDrive/testing/csv_files/test_labels.csv"
IMAGE_DIR = "/content/drive/MyDrive/testing/cards_in_box/images"
TARGET_COL = "class"
TRAIN_BATCH_SIZE = 1
VALID_BATCH_SIZE = 1
TRAIN_WORKERS = 1
LEARNING_RATE = 2e-4
EPOCHS = 10
NUM_CLASSES = 3
DETECTION_THRESHOLD = 0.6

BACKBONE = "mobile_net"
MODEL_SAVE_PATH = "/content/drive/MyDrive/testing/model/faster_rcnn_version1_{}.pt".format(BACKBONE)

OUTPUT_PATH = "/content/drive/MyDrive/testing/cards_in_box/output"

PREDICT_IMAGE = '/content/predicting_images/GOPR1561.JPG'
SAVE_IMAGE = '/content/saving_images'
SAVE_DIR = "/content/drive/MyDrive/testing/cards_in_box/output"

Dataset

-----

In [ ]:
import os
import numpy as np
import cv2
import torch
from torch.utils.data import DataLoader, Dataset

__all__ = ["detection_dataset"]


class detection_dataset(Dataset):
    def __init__(self, dataframe, image_dir, target, transforms=None, train=True):
        super().__init__()

        self.image_ids = dataframe["image_id"].unique()
        self.image_dir = image_dir
        self.transforms = transforms
        self.df = dataframe
        self.train = train
        self.target = target

    def __len__(self):
        return self.image_ids.shape[0]

    def __getitem__(self, index):
        image_id = self.image_ids[index]
        image_src = os.path.join(self.image_dir, str(image_id))
        image = cv2.imread(image_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        if self.transforms is not None:  # Apply transformation
            image = self.transforms(image)

        if self.train is False:  # For test data
            return image, image_id

        # Else for train and validation data
        records = self.df[self.df["image_id"] == image_id]
        boxes = records[["xmin", "ymin", "xmax", "ymax"]].values
        boxes = torch.as_tensor(boxes, dtype=torch.float32)

        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)

        # For has_mask
        labels = records[self.target].values
        labels = torch.as_tensor(labels, dtype=torch.int64)
        # print(labels)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = torch.tensor([index])
        target["area"] = area

        return image, target, image_id


Engine training functions

-----

In [ ]:
import torch
import torchvision
import torchvision.transforms as T
import numpy as np
import time
from tqdm import tqdm

__all__ = ["train_fn", "eval_fn"]


def train_fn(train_dataloader, detector, optimizer, device, scheduler=None):
    detector.train()
    for images, targets, image_ids in train_dataloader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = detector(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if scheduler is not None:
            scheduler.step()
return loss_value


def eval_fn(val_dataloader, detector, device, detection_threshold=0.45):
    results = []
    detector.eval()
    with torch.no_grad():
        for images, targets, image_ids in val_dataloader:
            images = list(image.to(device) for image in images)

            model_time = time.time()
            outputs = detector(images)
            model_time = time.time() - model_time

            for i, image in enumerate(images):
                boxes = (
                    outputs[i]["boxes"].data.cpu().numpy()
                )  # Format of the output's box is [Xmin,Ymin,Xmax,Ymax]
                scores = outputs[i]["scores"].data.cpu().numpy()
                labels = outputs[i]["labels"].data.cpu().numpy()
                image_id = image_ids[i]
                result = {  # Store the image id and boxes and scores in result dict.
                    "image_id": image_id,
                    "boxes": boxes,
                    "scores": scores,
                    "labels": labels,
                }
                results.append(result)

    return results


Utils

-----

In [ ]:
import pandas as pd
import numpy as np
import torch
import cv2
from collections import defaultdict, deque
import datetime
import pickle
import time
import matplotlib.pyplot as plt
import torch.distributed as dist
import random
import errno
import os

# My utils
class AverageMeter:
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def show_loader_images(images, targets, device):
    images = list(image for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    boxes = targets[4]["boxes"].cpu().numpy().astype(np.int32)
    # Torch takes channels first format we need to change to channels last
    sample = images[4].permute(1, 2, 0).cpu().numpy()

    fig, ax = plt.subplots(1, 1, figsize=(16, 8))

    for box in boxes:
        cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (220, 0, 0), 3)

    ax.set_axis_off()
    ax.imshow(sample)


def random_show_images(root_dir, df, no_images=5, fmt=".jpg"):

    random_image_l = random.sample(range(0, 500), no_images)
    # random_image_l = [1, 2, 3]

    for image_id in random_image_l:

        img_path = os.path.join(root_dir, str(image_id))
        img_path += fmt
        # print(img_path)
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        print("image no {}".format(image_id))
        # plt.imshow(image)
        plt.figure()
        plt.axis("off")

        for i, image_name in enumerate(df["image_id"]):
            # print(image_name)
            if str(image_name) == str(image_id):
                # print("yes")
                xtl = int(df["xmin"].iloc[i])
                ytl = int(df["ymin"].iloc[i])
                xbr = int(df["xmax"].iloc[i])
                ybr = int(df["ymax"].iloc[i])
                # print(xtl, ytl, xbr, ybr)
                has_helmet = str(df["has_helmet"].iloc[i]) + " helmet"
                has_mask = str(df["has_mask"].iloc[i]) + " mask"
                cv2.rectangle(image, (xtl, ytl), (xbr, ybr), color=(0, 255, 0))
                cv2.putText(
                    image,
                    has_helmet,
                    (xtl, ytl),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (36, 255, 12),
                    2,
                )
                cv2.putText(
                    image,
                    has_mask,
                    (xbr, ybr),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (36, 255, 12),
                    2,
                )

        plt.imshow(image)


# Let's get distrigbution stats for our labeled data
def get_distribution_column(df, column):
    print(df[column].value_counts())
    df[column].value_counts().sort_values().plot(kind="bar")

class SmoothedValue(object):
    """Track a series of values and provide access to smoothed values over a
    window or the global series average.
    """

    def __init__(self, window_size=20, fmt=None):
        if fmt is None:
            fmt = "{median:.4f} ({global_avg:.4f})"
        self.deque = deque(maxlen=window_size)
        self.total = 0.0
        self.count = 0
        self.fmt = fmt

    def update(self, value, n=1):
        self.deque.append(value)
        self.count += n
        self.total += value * n

    def synchronize_between_processes(self):
        """
        Warning: does not synchronize the deque!
        """
        if not is_dist_avail_and_initialized():
            return
        t = torch.tensor([self.count, self.total], dtype=torch.float64, device="cuda")
        dist.barrier()
        dist.all_reduce(t)
        t = t.tolist()
        self.count = int(t[0])
        self.total = t[1]

    @property
    def median(self):
        d = torch.tensor(list(self.deque))
        return d.median().item()

    @property
    def avg(self):
        d = torch.tensor(list(self.deque), dtype=torch.float32)
        return d.mean().item()

    @property
    def global_avg(self):
        return self.total / self.count

    @property
    def max(self):
        return max(self.deque)

    @property
    def value(self):
        return self.deque[-1]

    def __str__(self):
        return self.fmt.format(
            median=self.median,
            avg=self.avg,
            global_avg=self.global_avg,
            max=self.max,
            value=self.value,
        )


def all_gather(data):
    """
    Run all_gather on arbitrary picklable data (not necessarily tensors)
    Args:
        data: any picklable object
    Returns:
        list[data]: list of data gathered from each rank
    """
    world_size = get_world_size()
    if world_size == 1:
        return [data]

    # serialized to a Tensor
    buffer = pickle.dumps(data)
    storage = torch.ByteStorage.from_buffer(buffer)
    tensor = torch.ByteTensor(storage).to("cuda")

    # obtain Tensor size of each rank
    local_size = torch.tensor([tensor.numel()], device="cuda")
    size_list = [torch.tensor([0], device="cuda") for _ in range(world_size)]
    dist.all_gather(size_list, local_size)
    size_list = [int(size.item()) for size in size_list]
    max_size = max(size_list)

    tensor_list = []
    for _ in size_list:
        tensor_list.append(torch.empty((max_size,), dtype=torch.uint8, device="cuda"))
    if local_size != max_size:
        padding = torch.empty(
            size=(max_size - local_size,), dtype=torch.uint8, device="cuda"
        )
        tensor = torch.cat((tensor, padding), dim=0)
    dist.all_gather(tensor_list, tensor)

    data_list = []
    for size, tensor in zip(size_list, tensor_list):
        buffer = tensor.cpu().numpy().tobytes()[:size]
        data_list.append(pickle.loads(buffer))

    return data_list


def reduce_dict(input_dict, average=True):
    """
    Args:
        input_dict (dict): all the values will be reduced
        average (bool): whether to do average or sum
    Reduce the values in the dictionary from all processes so that all processes
    have the averaged results. Returns a dict with the same fields as
    input_dict, after reduction.
    """
    world_size = get_world_size()
    if world_size < 2:
        return input_dict
    with torch.no_grad():
        names = []
        values = []
        # sort the keys so that they are consistent across processes
        for k in sorted(input_dict.keys()):
            names.append(k)
            values.append(input_dict[k])
        values = torch.stack(values, dim=0)
        dist.all_reduce(values)
        if average:
            values /= world_size
        reduced_dict = {k: v for k, v in zip(names, values)}
    return reduced_dict


class MetricLogger(object):
    def __init__(self, delimiter="\t"):
        self.meters = defaultdict(SmoothedValue)
        self.delimiter = delimiter

    def update(self, **kwargs):
        for k, v in kwargs.items():
            if isinstance(v, torch.Tensor):
                v = v.item()
            assert isinstance(v, (float, int))
            self.meters[k].update(v)

    def __getattr__(self, attr):
        if attr in self.meters:
            return self.meters[attr]
        if attr in self.__dict__:
            return self.__dict__[attr]
        raise AttributeError(
            "'{}' object has no attribute '{}'".format(type(self).__name__, attr)
        )

    def __str__(self):
        loss_str = []
        for name, meter in self.meters.items():
            loss_str.append("{}: {}".format(name, str(meter)))
        return self.delimiter.join(loss_str)

    def synchronize_between_processes(self):
        for meter in self.meters.values():
            meter.synchronize_between_processes()

    def add_meter(self, name, meter):
        self.meters[name] = meter

    def log_every(self, iterable, print_freq, header=None):
        i = 0
        if not header:
            header = ""
        start_time = time.time()
        end = time.time()
        iter_time = SmoothedValue(fmt="{avg:.4f}")
        data_time = SmoothedValue(fmt="{avg:.4f}")
        space_fmt = ":" + str(len(str(len(iterable)))) + "d"
        log_msg = self.delimiter.join(
            [
                header,
                "[{0" + space_fmt + "}/{1}]",
                "eta: {eta}",
                "{meters}",
                "time: {time}",
                "data: {data}",
                "max mem: {memory:.0f}",
            ]
        )
        MB = 1024.0 * 1024.0
        for obj in iterable:
            data_time.update(time.time() - end)
            yield obj
            iter_time.update(time.time() - end)
            if i % print_freq == 0 or i == len(iterable) - 1:
                eta_seconds = iter_time.global_avg * (len(iterable) - i)
                eta_string = str(datetime.timedelta(seconds=int(eta_seconds)))
                print(
                    log_msg.format(
                        i,
                        len(iterable),
                        eta=eta_string,
                        meters=str(self),
                        time=str(iter_time),
                        data=str(data_time),
                        memory=torch.cuda.max_memory_allocated() / MB,
                    )
                )
            i += 1
            end = time.time()
        total_time = time.time() - start_time
        total_time_str = str(datetime.timedelta(seconds=int(total_time)))
        print(
            "{} Total time: {} ({:.4f} s / it)".format(
                header, total_time_str, total_time / len(iterable)
            )
        )


def collate_fn(batch):
    return tuple(zip(*batch))


def warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor):
    def f(x):
        if x >= warmup_iters:
            return 1
        alpha = float(x) / warmup_iters
        return warmup_factor * (1 - alpha) + alpha

    return torch.optim.lr_scheduler.LambdaLR(optimizer, f)


def mkdir(path):
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise


def setup_for_distributed(is_master):
    """
    This function disables printing when not in master process
    """
    import builtins as __builtin__

    builtin_print = __builtin__.print

    def print(*args, **kwargs):
        force = kwargs.pop("force", False)
        if is_master or force:
            builtin_print(*args, **kwargs)

    __builtin__.print = print


def is_dist_avail_and_initialized():
    if not dist.is_available():
        return False
    if not dist.is_initialized():
        return False
    return True


def get_world_size():
    if not is_dist_avail_and_initialized():
        return 1
    return dist.get_world_size()


def get_rank():
    if not is_dist_avail_and_initialized():
        return 0
    return dist.get_rank()


def is_main_process():
    return get_rank() == 0


def save_on_master(*args, **kwargs):
    if is_main_process():
        torch.save(*args, **kwargs)


def init_distributed_mode(args):
    if "RANK" in os.environ and "WORLD_SIZE" in os.environ:
        args.rank = int(os.environ["RANK"])
        args.world_size = int(os.environ["WORLD_SIZE"])
        args.gpu = int(os.environ["LOCAL_RANK"])
    elif "SLURM_PROCID" in os.environ:
        args.rank = int(os.environ["SLURM_PROCID"])
        args.gpu = args.rank % torch.cuda.device_count()
    else:
        print("Not using distributed mode")
        args.distributed = False
        return

    args.distributed = True

    torch.cuda.set_device(args.gpu)
    args.dist_backend = "nccl"
    print(
        "| distributed init (rank {}): {}".format(args.rank, args.dist_url), flush=True
    )
    torch.distributed.init_process_group(
        backend=args.dist_backend,
        init_method=args.dist_url,
        world_size=args.world_size,
        rank=args.rank,
    )
    torch.distributed.barrier()
    setup_for_distributed(args.rank == 0)


Train

-----

In [ ]:
from pprint import pprint
from torchvision import transforms as T
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import time

train_loss= []

def run():
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    train_df = pd.read_csv(TRAIN_CSV_PATH)
    valid_df = pd.read_csv(VALIDATION_CSV_PATH)

    train_df=train_df[['filename','xmin','ymin','xmax','ymax','class']]
    train_df.columns=['image_id','xmin','ymin','xmax','ymax','class']
    train_df['class'] = train_df['class'].map({'green-card': 1, 'yellow-card' : 2 })



    valid_df=valid_df[['filename','xmin','ymin','xmax','ymax','class']]
    valid_df.columns=['image_id','xmin','ymin','xmax','ymax','class']
    valid_df['class'] = valid_df['class'].map({'green-card': 1, 'yellow-card' : 2 })

    
    train_dataset =detection_dataset(
        train_df,
        IMAGE_DIR,
        target=TARGET_COL,
        train=True,
        transforms=T.Compose([T.ToTensor()]),
    )

    valid_dataset = detection_dataset(
        valid_df,
        IMAGE_DIR,
        target=TARGET_COL,
        train=True,
        transforms=T.Compose([T.ToTensor()]),
    )

    # print(train_dataset)

    train_dataloader = DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        shuffle=False,
        collate_fn=collate_fn,
    )

    valid_dataloader = DataLoader(
        valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        shuffle=False,
        collate_fn=collate_fn,
    )

    print("Data Loaders created")

    detector = create_model(NUM_CLASSES, backbone=BACKBONE)
    params = [p for p in detector.parameters() if p.requires_grad]
    optimizer = optim.Adam(params, lr=LEARNING_RATE)
    # lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
    detector.to(device)

    print("Model loaded to device")

    print("---------------- Training Started --------------")

    for epoch in range(EPOCHS):
      t = time.time()
      loss_value = train_fn(train_dataloader, detector, optimizer, device)
      t = time.time()-t
      min = int(t/60)
      print("epoch = {}, time_needed = {} mins {:.2f} secs ,Training_loss = {}".format(epoch+1,min ,t - min * 60, loss_value))
      train_loss.append(loss_value)  
      # Set the threshold as per needs
      results = eval_fn(
      valid_dataloader,
      detector,
      device,
      detection_threshold=DETECTION_THRESHOLD,
      )
      # Pretty printing the results
      # pprint(results)
    torch.save(detector.state_dict(), MODEL_SAVE_PATH)
    print("-" * 25)
    print("Model Trained and Saved to Disk")



if __name__ == "__main__":
    run()


Inference

----

In [ ]:
# This script does only inference from the loaded model
import cv2
import matplotlib.pyplot as plt
import torch
import os
from PIL import Image
import torchvision.transforms as T
from google.colab.patches import cv2_imshow
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
!rm -r /content/predicting_images/.ipynb_checkpoints
__all__ = [
    "load_model",
    "load_image_tensor",
    "get_prediction",
    "draw_box",
    "load_image_to_plot",
    "save_prediction",
    "get_folder_results",
]


def load_model():
    detector = create_model(num_classes=NUM_CLASSES)
    detector.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=device))
    detector.eval()
    detector.to(device)
    return detector


# Load the detector for inference


def load_image_tensor(image_path, device):
    image_tensor = T.ToTensor()(Image.open(image_path))
    input_images = [image_tensor.to(device)]
    return input_images


def get_prediction(detector, images):
    with torch.no_grad():
        prediction = detector(images)
        return prediction


def draw_box(image, box, label_id, score):
    xmin = int(box[0])
    ymin = int(box[1])
    xmax = int(box[2])
    ymax = int(box[3])
    # Some hard coding for label
    if label_id == 1:
        label = "Green-Card" + ": {0:.2f}" .format(100 * score) + " %"
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color=(0, 255, 0),thickness = 5)
        cv2.putText(image, label, (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0),5)

    elif label_id == 2:
        label = "Yellow-Card"+ ": {0:.2f}" .format(100 * score) + " %"
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color=(0, 255, 255),thickness = 5)
        cv2.putText(image, label, (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,255),7)

    elif label_id == 3:
        label = "invisible"
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color=(0, 0, 255))
    elif label_id == 4:
        label = "wrong"
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color=(0, 0, 255))

    # cv2.putText(image, label, (xmax, ymax), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12),5)


def load_image_to_plot(image_dir):
    image = cv2.imread(image_dir, cv2.IMREAD_COLOR)
    return image


def save_prediction(prediction, image_name, image):
    for pred in prediction:
        boxes = pred["boxes"].data.cpu().numpy()
        labels = pred["labels"].data.cpu().numpy()
        scores = pred["scores"].data.cpu().numpy()

    for i in range(len(labels)):
        if scores[i] >0.90: #DETECTION_THRESHOLD:
            box_draw = boxes[i]
            label_draw = labels[i]  
            score = scores[i]
            draw_box(image, box_draw, label_draw, score)
    cv2_imshow(image)
    #cv2.imwrite(image_name, image)


def get_folder_results(detector, image_dir, device):
    for image in os.listdir(image_dir):
        image_path = os.path.join(image_dir, image)
        input_images = load_image_tensor(image_path, device)
        prediction = get_prediction(detector, input_images)
        print(prediction)
        image_loaded = load_image_to_plot(image_path)
        save_path = os.path.join(SAVE_DIR, image)
        save_prediction(prediction, save_path, image_loaded)


if __name__ == "__main__":
   detector = load_model()
   print("---------- Model succesfully loaded -------- ")
   image_dir = '/content/predicting_images'